In [1]:
import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import time
import calendar
from datetime import date
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException
import ipyparallel as ipp

In [2]:
products=['Laptop']

In [3]:
url='https://www.flipkart.com/'
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get(url)

In [4]:
try:
    button_x=driver.find_elements_by_xpath('/html/body/div[2]/div/div/button')
    for i in range(len(button_x)):
        if button_x[i].text=='✕':
            button_x[i].click() 
except:
    pass

In [5]:
ratings_5=[]
ratings_4=[]
ratings_3=[]
ratings_2=[]
ratings_1=[]

In [6]:
def change_product(i):
    search=driver.find_element_by_xpath("//input[@title='Search for products, brands and more']")
    search_btn=driver.find_element_by_class_name('L0Z3Pu')
    search.clear()
    search.send_keys(i)
    search_btn.click()

In [7]:
def find_pages(page):
    page_tag=driver.find_elements_by_xpath('//nav[@class="yFHi8N"]//a')
    for pagy in page_tag:
        if pagy.get_attribute('href') not in page:
            page.append(pagy.get_attribute('href'))
    return page[:-1]

In [8]:
def find_product_links():
    linkss=[]
    urls=driver.find_elements_by_xpath('//div[@class="_2kHMtA"]//a[@class="_1fQZEK"]')
    for u in urls:
        linkss.append(u.get_attribute('href'))
    linkss=linkss[:-1]
    return linkss

In [9]:
def item_review_nav(next_page):
    next_page_tag=driver.find_elements_by_xpath('//nav[@class="yFHi8N"]//a')
    for nexty in next_page_tag:
        if nexty.get_attribute('href') not in next_page:
            next_page.append(nexty.get_attribute('href'))
    return next_page

In [10]:
def populate_data(rating_5,rating_4,rating_3,rating_2,rating_1):
    iteration=driver.find_elements_by_xpath('//div[@class="_1YokD2 _3Mn1Gg col-9-12"]/div[@class="_1AtVbE col-12-12"]/div[@class="_27M-vq"]/div/div/div[@class="row"][1]')
    for iterr in iteration:
        if len(iterr.text.split())>=2:
            if iterr.text.split()[0]=='5' and len(rating_5)<2500:
                rating_5.append(' '.join(iterr.text.split()[1:]))
            elif iterr.text.split()[0]=='4' and len(rating_4)<2500:
                rating_4.append(' '.join(iterr.text.split()[1:]))
            elif iterr.text.split()[0]=='3' and len(rating_3)<2500:
                rating_3.append(' '.join(iterr.text.split()[1:]))
            elif iterr.text.split()[0]=='2' and len(rating_2)<2500:
                rating_2.append(' '.join(iterr.text.split()[1:]))
            elif iterr.text.split()[0]=='1' and len(rating_1)<2500:
                rating_1.append(' '.join(iterr.text.split()[1:]))
    return rating_5,rating_4,rating_3,rating_2,rating_1

In [11]:
for i in products:
    print('*'*20)
    print(i)
    print('*'*20)
    change_product(i)
    rating_5=[]
    rating_4=[]
    rating_3=[]
    rating_2=[]
    rating_1=[]
    page=[]
    time.sleep(2)
    while len(rating_1)<2500 or len(rating_2)<2500 or len(rating_3)<2500 or len(rating_4)<2500 or len(rating_5)<2500:
        page=find_pages(page)
        for pagy in page:
            driver.get(pagy)
            time.sleep(2)
            links=find_product_links()
            for linky in links:
                if len(rating_1)>=2500 and len(rating_2)>=2500 and len(rating_3)>=2500 and len(rating_4)>=2500 and len(rating_5)>=2500:
                    break
                driver.get(linky)
                time.sleep(2)
                try:
                    button_read_more=driver.find_element_by_xpath('//div[@class="_3UAT2v _16PBlm"]')
                    button_read_more.click()
                    time.sleep(2)
                    next_page=[]
                    next_page=item_review_nav(next_page)
                    if len(next_page)==0:
                        next_page.append(linky)
                    if len(next_page)>9:
                        last_updated=len(next_page)
                        driver.get(next_page[-1])
                        time.sleep(2)
                        next_page=item_review_nav(next_page)
                        while last_updated!=len(next_page):
                            last_updated=len(next_page)
                            driver.get(next_page[-1])
                            time.sleep(2)
                            next_page=item_review_nav(next_page)
                    for nextup in next_page:
                        driver.get(nextup)
                        time.sleep(2)
                        rating_5,rating_4,rating_3,rating_2,rating_1=populate_data(rating_5,rating_4,rating_3,rating_2,rating_1)
                        if len(rating_1)>=2500 and len(rating_2)>=2500 and len(rating_3)>=2500 and len(rating_4)>=2500 and len(rating_5)>=2500:
                            break
                except:
                    pass
            print([len(rating_1),len(rating_2),len(rating_3),len(rating_4),len(rating_5)])
            if len(rating_1)>=2500 and len(rating_2)>=2500 and len(rating_3)>=2500 and len(rating_4)>=2500 and len(rating_5)>=2500:
                break
    driver.get(url)
    time.sleep(2)
    ratings_1.extend(rating_1)
    ratings_2.extend(rating_2)
    ratings_3.extend(rating_3)
    ratings_4.extend(rating_4)
    ratings_5.extend(rating_5)

********************
Laptop
********************
[93, 32, 93, 343, 918]
[153, 51, 170, 606, 1813]
[238, 88, 277, 952, 2500]
[346, 121, 381, 1364, 2500]
[411, 147, 454, 1646, 2500]
[485, 170, 540, 1949, 2500]
[541, 186, 595, 2143, 2500]
[634, 221, 693, 2401, 2500]
[745, 251, 766, 2500, 2500]
[823, 280, 846, 2500, 2500]
[890, 302, 931, 2500, 2500]
[979, 337, 1037, 2500, 2500]
[1084, 365, 1139, 2500, 2500]
[1522, 495, 1402, 2500, 2500]
[1828, 547, 1533, 2500, 2500]
[1904, 567, 1597, 2500, 2500]
[1994, 601, 1694, 2500, 2500]
[2090, 633, 1786, 2500, 2500]
[2152, 657, 1869, 2500, 2500]
[2249, 687, 1975, 2500, 2500]
[2289, 706, 2037, 2500, 2500]
[2356, 733, 2108, 2500, 2500]
[2416, 750, 2158, 2500, 2500]
[2446, 759, 2189, 2500, 2500]
[2500, 788, 2258, 2500, 2500]
[2500, 788, 2258, 2500, 2500]
[2500, 788, 2258, 2500, 2500]
[2500, 788, 2258, 2500, 2500]
[2500, 788, 2258, 2500, 2500]
[2500, 824, 2374, 2500, 2500]
[2500, 835, 2422, 2500, 2500]
[2500, 858, 2500, 2500, 2500]
[2500, 914, 2500, 2500,

In [12]:
laptop_5=pd.DataFrame({})
laptop_5['comments']=ratings_5[:2500]
laptop_5['rating']=[5 for i in range(2500)]
laptop_5

,comments,rating
0,Must buy!,5
1,Great product,5
2,Best in the market!,5
3,Worth every penny,5
4,Wonderful,5
...,...,...
2495,Great product,5
2496,Super!,5
2497,Mind-blowing purchase,5
2498,Super!,5


In [13]:
laptop_4=pd.DataFrame({})
laptop_4['comments']=ratings_4[:2500]
laptop_4['rating']=[4 for i in range(2500)]
laptop_4

,comments,rating
0,Good quality product,4
1,Nice product,4
2,Wonderful,4
3,Wonderful,4
4,Value-for-money,4
...,...,...
2495,Wonderful,4
2496,Very Good,4
2497,Good quality product,4
2498,Nice product,4


In [14]:
laptop_3=pd.DataFrame({})
laptop_3['comments']=ratings_3[:2500]
laptop_3['rating']=[3 for i in range(2500)]
laptop_3

,comments,rating
0,Fair,3
1,Nice,3
2,Good,3
3,Does the job,3
4,Good,3
...,...,...
2495,Fair,3
2496,Fair,3
2497,Fair,3
2498,Just okay,3


In [15]:
laptop_2=pd.DataFrame({})
laptop_2['comments']=ratings_2[:2500]
laptop_2['rating']=[2 for i in range(2500)]
laptop_2

,comments,rating
0,Could be way better,2
1,Bad quality,2
2,Expected a better product,2
3,Expected a better product,2
4,Moderate,2
...,...,...
2495,Moderate,2
2496,Slightly disappointed,2
2497,Moderate,2
2498,Could be way better,2


In [16]:
laptop_1=pd.DataFrame({})
laptop_1['comments']=ratings_1[:2500]
laptop_1['rating']=[1 for i in range(2500)]
laptop_1

,comments,rating
0,Worst experience ever!,1
1,Useless product,1
2,Worthless,1
3,Horrible,1
4,Terrible product,1
...,...,...
2495,Very poor,1
2496,Not recommended at all,1
2497,Waste of money!,1
2498,Utterly Disappointed,1


In [17]:
flipkart_laptop=laptop_5.copy()

In [18]:
flipkart_laptop=flipkart_laptop.append(laptop_4,ignore_index = True)
flipkart_laptop=flipkart_laptop.append(laptop_3,ignore_index = True)
flipkart_laptop=flipkart_laptop.append(laptop_2,ignore_index = True)
flipkart_laptop=flipkart_laptop.append(laptop_1,ignore_index = True)
flipkart_laptop

,comments,rating
0,Must buy!,5
1,Great product,5
2,Best in the market!,5
3,Worth every penny,5
4,Wonderful,5
...,...,...
12495,Very poor,1
12496,Not recommended at all,1
12497,Waste of money!,1
12498,Utterly Disappointed,1


In [19]:
flipkart_laptop.to_csv('flipkart_laptop.csv',index=False)